# Data Collection

## Objectives

* Fetch data from Kaggle and prepare it for further processes.

## Inputs

* A Kaggle JSON file - the authentication token. 

## Outputs

* Generate Dataset: inputs/datasets/mildew_dataset.

## Additional Comments

* No additional comments.



---

# Import packages

In [ ]:
import os

# Change working directory

We need to change the working directory from its current folder to its parent folder

In [ ]:
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory

In [ ]:
os.chdir('/workspace/project-portfolio-5')
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Install Kaggle

---

# Data Preparation

---

# Conclusions and Next Steps